In [1]:
import sys

sys.path.append("/gstore/project/paneldesign/pkg")

sys.path.append("/gstore/home/taol9/gitsss/yo/tao")

from Bio import SeqIO
from Bio.Seq import Seq
from Bio import Entrez
import csv
from collections import defaultdict,OrderedDict
from collections import Counter
import matplotlib.pyplot as plt

import pandas as pd
import os
import pickle
import random
from myfunc import rc, RandomDNA


In [2]:
# !pip3 install --target /gstore/project/paneldesign/pkg barcode

In [3]:
def bgen(length,maxstretch=3,distance=3,):
    candi=[]
    for i in range(30000):
        x=str(RandomDNA(length))
        if 45<GC(x)<55:
            candi.append(x)
    candi=barcode.filter_stretches(candi, maxstretch)
    candi=barcode.filter_distance(candi, distance)
    return candi

# TSO(25nt)-flankingsequence(25nt)-Index1-UMI-repeat1-spacer(4nt)-repeat2-spacer-repeat3-spacer-repeat4-...repeatX-polyA;

## define adaptor


In [4]:
BC001 = "ACTTTAGG"
BC002 = "AACGGGAA"
BC003 = "AGTAGGCT"
BC004 = "ATGTTGAC"
BC005 = "ACAGACCT"
BC006 = "ATCCCAAC"
BC007 = "AAGTAGAG"
BC008 = "AGCTGTGA"
BC009 = "ACAGTCTG"
BC010 = "AGTGAGTG"
BC011 = "AGAGGCAA"
BC012 = "ACTACTCA"
BC013 = "ATACGTCA"
BC014 = "ATCATGTG"
BC015 = "AACGCCGA"
BC016 = "ATTCGGTT"

Mly1_R_w='GCTTCCTGATGAGTCCGATG'
Mly1_F_w='GCAACGACTCCACACTCATA'

# TSO(25nt)-flankingsequence(25nt)-Index1-UMI-repeat1-spacer(4nt)-repeat2-spacer-repeat3-spacer-repeat4-...repeatX-polyA;

In [5]:
# random combination
# spacer=RandomDNA(4)
# units=["C","TG","AG"]
# str_len=[9,18]
# UMI="N"*6
# index=BC003
# TSO='AAGCAGTGGTATCAACGCAGAGTACAT'
# dialout_adaptor_F=Mly1_F_w='GCAACGACTCCACACTCATA'
# dialout_adaptor_R=Mly1_R_w='GCTTCCTGATGAGTCCGATG'
# l=TSO+dialout_adaptor_F+index+UMI
# n=0
# while len(l)<1500:
#     l=l+random.choice(str_len)*random.choice(units)+spacer
#     n=n+1
# #     print (l)
# print(n)
# l=l+dialout_adaptor_R


    

    
    

In [6]:
#fix length with units combination
def strtracer(index,maxlen=2000,str_len=15,UMIlen=10):
    units=["AC","C","AG"]
    UMI="N"*UMIlen
    TSO='AAGCAGTGGTATCAACGCAGAGTACAT'
    dialout_adaptor_F=Mly1_F_w='GCAACGACTCCACACTCATA'
    dialout_adaptor_R=Mly1_R_w='GCTTCCTGATGAGTCCGATG'
    bridge=RandomDNA(25)
    l=partA=TSO+dialout_adaptor_F+index+UMI+bridge
    n=0
    while len(l)<maxlen-len(partA):
        l=l+str_len*random.choice(units)+RandomDNA(str_len)+str_len*random.choice(units)+RandomDNA(str_len)+str_len*random.choice(units)+RandomDNA(str_len)
        n=n+3
#     print (l)
    print(n)
    l=str(l+dialout_adaptor_R)
    return {"index":index,"partA":str(partA), "partB":l[len(partA)-25:],"Full":str(l), "strs_counts":n,"len":len(l)}

In [7]:
indexes='''TGGCACTCAT
ATATGCGCCG
TTACCTACGG
CTTATCCGGA
GCTACTGACG
AGCCAATTGC
AAGTCGTAAC
CTCGGGCAAT
TCCATAGGTG
GAAGTTCATC'''.split()
indexes

['TGGCACTCAT',
 'ATATGCGCCG',
 'TTACCTACGG',
 'CTTATCCGGA',
 'GCTACTGACG',
 'AGCCAATTGC',
 'AAGTCGTAAC',
 'CTCGGGCAAT',
 'TCCATAGGTG',
 'GAAGTTCATC']

In [8]:
output=dict()
for idx in indexes[:4]:
    output["2K_"+str(idx)]=strtracer(index=idx,maxlen=2000,str_len=15,UMIlen=10)
for idx in indexes[-4:]:
    output["5K_"+str(idx)]=strtracer(index=idx,maxlen=5000,str_len=15,UMIlen=10)

45
48
48
45
123
123
123
126


In [9]:

df=pd.DataFrame.from_dict(output,orient='index')


In [10]:
df

,index,partA,partB,Full,strs_counts,len
2K_ATATGCGCCG,ATATGCGCCG,AAGCAGTGGTATCAACGCAGAGTACATGCAACGACTCCACACTCAT...,GTAGTCCGAGCGTCCTGTGCGCCGTACACACACACACACACACACA...,AAGCAGTGGTATCAACGCAGAGTACATGCAACGACTCCACACTCAT...,48,1987
2K_CTTATCCGGA,CTTATCCGGA,AAGCAGTGGTATCAACGCAGAGTACATGCAACGACTCCACACTCAT...,GGCGCCCATGGTGTGATTAACGCAAAGAGAGAGAGAGAGAGAGAGA...,AAGCAGTGGTATCAACGCAGAGTACATGCAACGACTCCACACTCAT...,45,1987
2K_TGGCACTCAT,TGGCACTCAT,AAGCAGTGGTATCAACGCAGAGTACATGCAACGACTCCACACTCAT...,AAGACTATACAGCGTAACGCAAGGAACACACACACACACACACACA...,AAGCAGTGGTATCAACGCAGAGTACATGCAACGACTCCACACTCAT...,45,1942
2K_TTACCTACGG,TTACCTACGG,AAGCAGTGGTATCAACGCAGAGTACATGCAACGACTCCACACTCAT...,CAGTAATTTAAAAGAGGTCTAGTCAAGAGAGAGAGAGAGAGAGAGA...,AAGCAGTGGTATCAACGCAGAGTACATGCAACGACTCCACACTCAT...,48,1972
5K_AAGTCGTAAC,AAGTCGTAAC,AAGCAGTGGTATCAACGCAGAGTACATGCAACGACTCCACACTCAT...,ACGCACAAGCCCAACTGCCGTCGTCCCCCCCCCCCCCCCCTGACGG...,AAGCAGTGGTATCAACGCAGAGTACATGCAACGACTCCACACTCAT...,123,4972
5K_CTCGGGCAAT,CTCGGGCAAT,AAGCAGTGGTATCAACGCAGAGTACATGCAACGACTCCACACTCAT...,TTCCGCCTTCATGAGGAATCGGCTGACACACACACACACACACACA...,AAGCAGTGGTATCAACGCAGAGTACATGCAACGACTCCACACTCAT...,123,5032
5K_GAAGTTCATC,GAAGTTCATC,AAGCAGTGGTATCAACGCAGAGTACATGCAACGACTCCACACTCAT...,TTTGACATGCACTAAATATATGTTTCCCCCCCCCCCCCCCGTACCG...,AAGCAGTGGTATCAACGCAGAGTACATGCAACGACTCCACACTCAT...,126,5047
5K_TCCATAGGTG,TCCATAGGTG,AAGCAGTGGTATCAACGCAGAGTACATGCAACGACTCCACACTCAT...,AGAGCCTGAGGCAAGTGATGCCGTAACACACACACACACACACACA...,AAGCAGTGGTATCAACGCAGAGTACATGCAACGACTCCACACTCAT...,123,4942


In [11]:
df.to_csv('str_tracer_230314.csv')

In [12]:
pwd

'/gstore/home/taol9/gitsss/yo/strlineage'